In [1]:
import numpy as np

import chugunov_indicator as chug

import pynucastro as pyna
import yt

In [2]:
# Double Detonation data: http://groot.astro.sunysb.edu/common/
ds = yt.load('./data/subch_plt17526/')
ad = ds.all_data()

yt : [INFO     ] 2024-10-08 23:20:48,596 Parameters: current_time              = 0.800001438968278
yt : [INFO     ] 2024-10-08 23:20:48,597 Parameters: domain_dimensions         = [ 640 1280    1]
yt : [INFO     ] 2024-10-08 23:20:48,598 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-10-08 23:20:48,599 Parameters: domain_right_edge         = [5.12000000e+09 1.02400000e+10 6.28318531e+00]


In [3]:
reaclib_library = pyna.ReacLibLibrary()

nuclei = [field[2:-1] for field in np.array(ds.field_list)[:,1] if "X(" in field]
comp = pyna.Composition(nuclei)

mynet = reaclib_library.linking_nuclei(comp.keys())
pynet = pyna.PythonNetwork(libraries=[mynet])

screen_map = pyna.screening.get_screening_map(
    pynet.get_rates(),
    symmetric_screening=pynet.symmetric_screening
)

In [4]:
comp.set_solar_like()
plasma = pyna.make_plasma_state(1e8, 1e4, comp.get_molar())

In [5]:
pair = screen_map[5]
scn_fac = pyna.make_screen_factors(pair.n1, pair.n2)

In [6]:
%%timeit

F = pyna.screening.chugunov_2009(plasma, scn_fac)

The slowest run took 12.32 times longer than the fastest. This could mean that an intermediate result is being cached.
7.56 µs ± 9.53 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit

skip = chug.skip_screening(plasma, scn_fac)

1.34 µs ± 32.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


This means that for individual screening factor calculations, the current model is a little under twice as fast as the other function.

To test how effective this, we can test more double detonation data.

In [8]:
# Double Detonation data: http://groot.astro.sunysb.edu/common/
ds = yt.load('./data/subch_plt21866/')
ad = ds.all_data()

yt : [INFO     ] 2024-10-08 23:21:10,355 Parameters: current_time              = 1.00002871903213
yt : [INFO     ] 2024-10-08 23:21:10,356 Parameters: domain_dimensions         = [ 640 1280    1]
yt : [INFO     ] 2024-10-08 23:21:10,356 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-10-08 23:21:10,357 Parameters: domain_right_edge         = [5.12000000e+09 1.02400000e+10 6.28318531e+00]


In [9]:
Xs = (np.array([ad[f"X({nucleus})"] for nucleus in nuclei]).T)
T, D = ad["Temp"], ad["density"]

In [10]:
rng = np.random.default_rng(10)

total = 10000
choices = rng.choice(np.arange(len(ad["density"])), total)
choices

array([3123643, 3845543, 1062025, ...,  929823,  784546,  171927])

In [11]:
plasma_states = []
for i in choices:
    comp.set_array(Xs[i])
    plasma = pyna.make_plasma_state(T[i], D[i], comp.get_molar())
    plasma_states.append(plasma)

In [12]:
%%timeit

for plasma_ in plasma_states:
    if chug.skip_screening(plasma, scn_fac):
        F = 1
    else:
        F = pyna.screening.chugunov_2009(plasma_, scn_fac)

13.7 ms ± 340 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%%timeit

for plasma_ in plasma_states:
    F = pyna.screening.chugunov_2009(plasma_, scn_fac)

18.3 ms ± 1.11 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


As such, this can significantly improve screening calculation times when used in `pynucastro`.